TODO
- [ ] wordcloud
- [ ] graphs, charts, statistics
- [ ] dalex

In [2]:
import bs4
import requests
import pandas as pd
import re
import fandom
import random
import time

In [3]:
domain_address = 'https://rickandmorty.fandom.com/'
all_pages_subpage = '/wiki/Special:AllPages'

In [6]:
# next_page_link = domain_address + all_pages_subpage


# list_of_pages = []

# # collect all links
# count = 0

# re_next = re.compile(r'Next page')

# while count < 5:
#     response = requests.get(next_page_link)
#     parsed = bs4.BeautifulSoup(response.text)

#     div_with_list_of_links = parsed.find_all('div', {'class':'mw-allpages-body'})[0]

#     for a_element in div_with_list_of_links.find_all('a', href=True):
#         list_of_pages.append(a_element['href'])
    
#     next_page_div = parsed.find_all('div', {'class':'mw-allpages-nav'})[0]
#     page_routes = next_page_div.find_all('a', href=True)
#     for page_route in page_routes:
#         if re.search(re_next, page_route.contents[0]):
#             next_page_route = page_route['href']
#         print(next_page_routes)
#     next_page_link = None
#     next_page_link =domain_address +  next_page_div.find_all('a', href=True)[-1]['href']
#     print(next_page_link)

#     count +=1
    

[<a href="/wiki/Special:AllPages?from=Charred+Morty" title="Special:AllPages">Next page (Charred Morty)</a>]
https://rickandmorty.fandom.com//wiki/Special:AllPages?from=Charred+Morty
[<a href="/wiki/Special:AllPages?from=%22Lawyer%22+Morty" title="Special:AllPages">Previous page ("Lawyer" Morty)</a>, <a href="/wiki/Special:AllPages?from=Female+Jerry" title="Special:AllPages">Next page (Female Jerry)</a>]
https://rickandmorty.fandom.com//wiki/Special:AllPages?from=Female+Jerry
[<a href="/wiki/Special:AllPages?from=Charred+Morty" title="Special:AllPages">Previous page (Charred Morty)</a>, <a href="/wiki/Special:AllPages?from=Jerry+Smith+%28C-137%29" title="Special:AllPages">Next page (Jerry Smith (C-137))</a>]
https://rickandmorty.fandom.com//wiki/Special:AllPages?from=Jerry+Smith+%28C-137%29
[<a href="/wiki/Special:AllPages?from=Female+Jerry" title="Special:AllPages">Previous page (Female Jerry)</a>, <a href="/wiki/Special:AllPages?from=Mr.+Goldenfold+%28Wasp+Universe%29" title="Special

Scrapping the links of all subpages in a fandom

In [6]:
# Backup cell

next_page_link = domain_address + all_pages_subpage


list_of_pages = []

# collect all links
count = 0

re_next = re.compile(r'Next page')

while count < 20:
    response = requests.get(next_page_link)
    parsed = bs4.BeautifulSoup(response.text)

    div_with_list_of_links = parsed.find_all('div', {'class':'mw-allpages-body'})[0]

    for a_element in div_with_list_of_links.find_all('a', href=True):
        list_of_pages.append(domain_address + a_element['href'])
    
    next_page_div = parsed.find_all('div', {'class':'mw-allpages-nav'})[0]
    page_routes = next_page_div.find_all('a', href=True)
    next_page_route = None
    for page_route in page_routes:
        if re.search(re_next, page_route.contents[0]):
            next_page_route = page_route['href']
    if next_page_route is None:
        break
    next_page_link =domain_address +  next_page_route
    print(next_page_link)

    count +=1
    

https://rickandmorty.fandom.com//wiki/Special:AllPages?from=Charred+Morty
https://rickandmorty.fandom.com//wiki/Special:AllPages?from=Female+Jerry
https://rickandmorty.fandom.com//wiki/Special:AllPages?from=Jerry+Smith+%28C-137%29
https://rickandmorty.fandom.com//wiki/Special:AllPages?from=Mr.+Goldenfold+%28Wasp+Universe%29
https://rickandmorty.fandom.com//wiki/Special:AllPages?from=Rainbow+Suspenders+Guy
https://rickandmorty.fandom.com//wiki/Special:AllPages?from=Samantha
https://rickandmorty.fandom.com//wiki/Special:AllPages?from=The+Non-Canonical+Adventures


In [306]:
len(list_of_pages)

2705

In [7]:
list_of_pages[3]

'https://rickandmorty.fandom.com//wiki/2001:_A_Space_Odyssey'

Scrapping text from all pages in the fandom

In [23]:
fandom.set_wiki('rickandmorty')
fp = fandom.page(title='2001:_A_Space_Odyssey')

In [24]:
fp.plain_text

"2001: A Space Odyssey\n2001: A Space Odyssey is a claymation short from Rick and Morty's The Non-Canonical Adventures.\nOverview\nThis short is a parody of the 1968 film, 2001: A Space Odyssey.\nPlot\nMorty is seen in a Space Shuttle wearing a Space suit. He asks Rick to open the door. Rick says that he can't do that. Morty asks what is the problem but Rick replies that this conversation can serve no purpose and tells Morty Goodbye in multiple languages.\nCharacters\nMorty Smith\nRick Sanchez (voice)"

In [41]:
def get_text_from_page(link:str)-> str:
    response = requests.get(link)
    parsed = bs4.BeautifulSoup(response.text)

    title = link.split('/')[-1]
    page = fandom.page(title=title)
    
    content = page.plain_text
    
    return content
    

In [219]:
get_text_from_page(list_of_pages[-2])

'Zombodians\nZombodians are an alien species that appears in the Rick and Morty universe. They appear throughout the show. One such member of this race is Gar Gloonch\nAppearance\nZombodians are shorter in height that the average human, ranging around three feet tall. They have a mix of light blue and seafoam green skin with two eyes with eyestalks that bend down to the face. Their blood color is orange.\nSection heading\nWrite the second section of your page here.'

In [43]:
articles = []
for page_link in list_of_pages[:10]:
    try:
        try:
            content = get_text_from_page(page_link)
            articles.append((page_link, content))
        except ValueError as ve:
            print(f'{page_link} article is empty')
    except fandom.error.PageError as pe:
        print(f'cannot find: {page_link}')


cannot find: https://rickandmorty.fandom.com//wiki/%22Lawyer%22_Morty
cannot find: https://rickandmorty.fandom.com//wiki/1%25_Visible
https://rickandmorty.fandom.com//wiki/A_Matter_of_Rick_or_Death article is empty
https://rickandmorty.fandom.com//wiki/A_Rick_and_His_Morty article is empty
https://rickandmorty.fandom.com//wiki/A_Rick_and_His_Morty_(Quest) article is empty


In [28]:
list_of_pages[:10]

['https://rickandmorty.fandom.com//wiki/%22Lawyer%22_Morty',
 'https://rickandmorty.fandom.com//wiki/1%25_Visible',
 'https://rickandmorty.fandom.com//wiki/1950s_Guy',
 'https://rickandmorty.fandom.com//wiki/2001:_A_Space_Odyssey',
 'https://rickandmorty.fandom.com//wiki/4th_Dimensional_Time_Cop_Headquarters',
 'https://rickandmorty.fandom.com//wiki/A_Bit_of_Pasta',
 'https://rickandmorty.fandom.com//wiki/A_Clockwork_Orange',
 'https://rickandmorty.fandom.com//wiki/A_Matter_of_Rick_or_Death',
 'https://rickandmorty.fandom.com//wiki/A_Rick_and_His_Morty',
 'https://rickandmorty.fandom.com//wiki/A_Rick_and_His_Morty_(Quest)']

In [33]:
articles

[('https://rickandmorty.fandom.com//wiki/1950s_Guy',
  '1950s Guy\n1950s Guy is a minor character from The Real Animated Adventures of Doc and Mharti\nAppearance\n1950s Guy wears a white t-shirt with the year "1955" written on it, with a black vest worn over it. He wears blue jeans and a black belt.\nHe is seen carrying a bag of tree seeds.\nHistory\n1950s guy was a character who appeared in The Real Animated Adventures of Doc and Mharti. In one of the shorts, Mharti McDonhalds is flying a kite, but it unfortunately gets stuck in a tree.\nDoc and Mharti travel back in time to the 1950s to prevent the tree from ever being planted. When the 1950s Guy arrives to plant the seed of the tree, Mharti licks Doc\'s balls. Once 1950s Guy sees this, he starts screaming in pain and eventually fades from the timeline, and is unable to plant the tree seed.\nLater, Mharti starts fading from the timeline in the present day, revealing that 1950\'s guy must be Mharti\'s great-great-great-great-great gra

- remove site navigation 
- remove non alphanumeric characters

In [288]:
df_articles = pd.DataFrame(articles, columns=['link', 'content'])


In [225]:
# df_articles.content = df_articles.content.str.lower()
# df_articles.content = df_articles.content.replace('\\n', value='', regex=True)

# df_articles.content = df_articles.content.replace('[^\w\s]', value=' ', regex=True)
# df_articles.content = df_articles.content.replace(to_replace='site navigation.*', value='', regex=True)

In [284]:
# df_articles.content = df_articles.content[0]
df_articles.content.iloc[0]

'1950s Guy\n1950s Guy is a minor character from The Real Animated Adventures of Doc and Mharti\nAppearance\n1950s Guy wears a white t-shirt with the year "1955" written on it, with a black vest worn over it. He wears blue jeans and a black belt.\nHe is seen carrying a bag of tree seeds.\nHistory\n1950s guy was a character who appeared in The Real Animated Adventures of Doc and Mharti. In one of the shorts, Mharti McDonhalds is flying a kite, but it unfortunately gets stuck in a tree.\nDoc and Mharti travel back in time to the 1950s to prevent the tree from ever being planted. When the 1950s Guy arrives to plant the seed of the tree, Mharti licks Doc\'s balls. Once 1950s Guy sees this, he starts screaming in pain and eventually fades from the timeline, and is unable to plant the tree seed.\nLater, Mharti starts fading from the timeline in the present day, revealing that 1950\'s guy must be Mharti\'s great-great-great-great-great grandfather.\nv • e Rick and Morty Characters \nMain chara

Now, as dataframe is clean, we perform NLP processing

In [207]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [214]:
stopwords


<WordListCorpusReader in '/home/lukik46/nltk_data/corpora/stopwords'>

In [289]:
def preprocess(raw_text:str):
    text = raw_text.lower()

    text = re.sub('\\nv(.|\n)*', ' ', text)
    text = re.sub('\\n', ' ', text)
    text = re.sub('[^\w\s]', ' ', text)
    # text = re.sub('site navigation.*', '', text)
    text = re.sub(' s ', '', text)

    # tokenize
    tokens = word_tokenize(text)

    # lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # drop stopwords
    # english_stopwords = stopwords.words('english')
    
    return ' '.join(tokens)

In [290]:
df_articles.content = df_articles.content.apply(preprocess)

In [291]:
df_articles.content.iloc[-2]

'a bit of pasta a bit of pasta is a song in the episode gotron jerrysis rickvangelion lyric hey hey mama miagotta plate of spaghetti for the takin papa piagotta great big robot that hemakin i got the mood and both of my eye you got ta bowl of pizza pie hey a bit of pasta with some anime combined yeah play that thang take it away boy a bit of pasta with some anime combined cause they re two genre but each one is intertwined yeah hey hey papa piagot a pan of lasagna in the oven mama miagotta monster that need a little shovin two dozen android form into one and we have only just begun hey a bit of pasta with some animе combined yeah anime combinеd anime combined anime combined anime combined a bit of pasta with some anime combined reference rick and morty official soundtrack a bit of pasta feat darren criss ryan elder rick and morty retrieved on september 19 2021 a bit of pasta feat darren criss ryan elder from rick and morty season 5 retrieved on september 19 2021 feat darren criss addit

Now the dataframe is ready to be processed by TfIdf

In [240]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [293]:
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2))

X = vectorizer.fit_transform(df_articles.content)


In [294]:
X.toarray().shape

(5, 551)

In [295]:
df = pd.DataFrame(X.toarray(),
    columns = vectorizer.get_feature_names_out())

In [296]:
df.head()

,19,19 2021,1950guy,1950guy mhartigreat,1950s,1950s guy,1950s prevent,1955,1955 written,1968,...,written,written black,yeah,yeah anime,yeah hey,yeah play,year,year 1955,zoom,zoom surrounded
0,0.000000,0.000000,0.050588,0.050588,0.354115,0.303527,0.050588,0.050588,0.050588,0.000000,...,0.050588,0.050588,0.000000,0.000000,0.000000,0.000000,0.050588,0.050588,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080808,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.076672,0.076672,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.115009,0.038336,0.038336,0.038336,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078498,0.078498


We can sort the idf vectors by frequency


In [297]:
idf_df = pd.DataFrame(
    vectorizer.idf_,
    index=vectorizer.get_feature_names_out(),
    columns=['idf_weights']
)

idf_df.sort_values(by=['idf_weights']).head(20)


,idf_weights
seen,1.405465
character,1.405465
smith,1.405465
adventure,1.405465
short,1.405465
film,1.405465
rick,1.405465
episode,1.405465
morty,1.405465
parody,1.693147


In [301]:
# compute cosine similarities  of each cluster]
from sklearn.metrics.pairwise import cosine_similarity

query = 'birdperson'

query_clean = preprocess(query)

query_vector = vectorizer.transform([query_clean])

In [302]:
query_vector.toarray().shape

(1, 551)

In [303]:
cosine_similarity(query_vector, X)

array([[0.        , 0.        , 0.        , 0.        , 0.15699589]])

In [305]:
df_articles.iloc[-1,-1]

'a clockwork orange a clockwork orange is a claymation short from rick and mortythe non canonical adventure series overview this short is a parody of the 1971 film a clockwork orange plot episode start with a shot at rickface he is drinking a the camera zoom out he is surrounded by friend birdperson mr meeseeks and mr poopybutthole they are drinking surrounded by furniture and statue that look like morty character rick sanchez birdperson mr meeseeks mr poopybutthole morty smith trvia rick is parody of alex delarge adult swim sign is seen behid the gang'

Dimensionality reduction